<a href="https://colab.research.google.com/github/Esaipriya/AISECT-ML/blob/main/LOG_REG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import pandas as pd
import numpy as np

In [2]:
dataset=pd.read_csv("/content/Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [5]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [6]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [7]:
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [9]:
from sklearn.linear_model import LogisticRegression

In [11]:
from sklearn.model_selection import GridSearchCV, cross_val_predict
param_grid={'penalty':['l2'], # 'l1', 'elasticnet' are not supported by the default solver 'lbfgs'
            'dual':[False], # dual=True is only supported for l2 penalty with liblinear solver
            'tol':[0.0001],
            'C':[1.0],
            'fit_intercept':[True],
            'random_state': [0, 42] # Provide valid integer values for random_state
            }
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X,y)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [1.0], 'dual': [False], 'fit_intercept': [True],
                         'penalty': ['l2'], 'random_state': [0, 42],
                         'tol': [0.0001]},
             scoring='f1_weighted', verbose=3)

In [12]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid.best_estimator_, X, y, cv=5)

In [13]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')

In [14]:
print("Best Parameters:", grid.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'C': 1.0, 'dual': False, 'fit_intercept': True, 'penalty': 'l2', 'random_state': 0, 'tol': 0.0001}
The f1_macro: 0.8161940298507463
The confusion Matrix:
 [[232  25]
 [ 47  96]]
The report:
               precision    recall  f1-score   support

           0       0.83      0.90      0.87       257
           1       0.79      0.67      0.73       143

    accuracy                           0.82       400
   macro avg       0.81      0.79      0.80       400
weighted avg       0.82      0.82      0.82       400



In [15]:
# Save Best Model
import pickle
filename = "LOG_REG_CV_best_model.sav"
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [16]:
# CV Results table
results_df = pd.DataFrame(grid.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_dual,param_fit_intercept,param_penalty,param_random_state,param_tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008998,0.005713,0.004643,0.000706,1.0,False,True,l2,0,0.0001,"{'C': 1.0, 'dual': False, 'fit_intercept': Tru...",0.629132,0.949546,0.936688,0.815688,0.697527,0.805716,0.127143,1
1,0.004301,0.000153,0.003667,0.000492,1.0,False,True,l2,42,0.0001,"{'C': 1.0, 'dual': False, 'fit_intercept': Tru...",0.629132,0.949546,0.936688,0.815688,0.697527,0.805716,0.127143,1


In [17]:
import numpy as np
import pickle
import pandas as pd  # Import pandas

# Load the saved model from the specified file
loaded_model = pickle.load(open('/content/LOG_REG_CV_best_model.sav', 'rb'))

# Get user input for age, estimated salary, and gender
age = int(input("Enter Age: "))
estimated_salary = int(input("Enter Estimated Salary: "))
gender = input("Enter Gender (Male/Female): ").lower()

# Convert gender input to numerical format (1 for Male, 0 for Female)
gender_male = 1 if gender == 'male' else 0

# Prepare the user input data as a pandas DataFrame with the correct feature names
# This matches the format the scaler was fitted on.
user_input_df = pd.DataFrame([[age, estimated_salary, gender_male]], columns=['Age', 'EstimatedSalary', 'Gender_Male'])


# Scale the user input using the same scaler 'sc' that was fitted on the training data (X)
# Note: It's crucial to use the same scaler fitted on the training data to scale new data
scaled_input = sc.transform(user_input_df)

# Make a prediction using the loaded model and the scaled user input
prediction = loaded_model.predict(scaled_input)

# Output the prediction result
if prediction[0] == 1:
    print("Purchased")
else:
    print("Not Purchased")

Enter Age: 67
Enter Estimated Salary: 30000
Enter Gender (Male/Female): female
Purchased
